# IdP-proxyのセットアップ

---

VCノードに [IdP-proxy](https://github.com/NII-cloud-operation/CoursewareHub-LC_idp-proxy)を構築する。

## はじめに

### 構成


CoursewareHubの構成要素を以下に示します。

![構成](images/cw-221-01.png)


このNotebookでは上図の `IdP-proxy` を構築します。

### 事前に準備が必要となるものについて

このNotebookを実行するにあたって、あらかじめ準備が必要となるものについて以下に記します。

* IdP-proxyのホスト名(FQDN)
* IdP-proxyのサーバ証明書、秘密鍵

## 操作対象の設定

操作対象となるAnsibleのグループ名を指定します。

### UnitGroup名

IdP-proxyの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# target_auth = 'IdPproxy'

target_auth = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

指定されたグループ名で対象となるノードが操作できることを確認します。

In [ ]:
!ansible {target_auth} -m ping

管理者権限での操作ができることを確認します。

In [ ]:
!ansible {target_auth} -b -a 'whoami'

## パラメータの設定

### ホスト名(FQDN)の指定

IdP-proxyのホスト名(FQDN)を指定してください。

In [ ]:
# (例)
# auth_fqdn = 'idpproxy.example.org'

auth_fqdn = 

 ### フェデレーション

参加するフェデレーションがテストフェデレーションであるか否かを次のセルで指定してください。

In [ ]:
# (例)
# enable_test_federation = False   # 運用フェデレーション
# enable_test_federation = True    # テストフェデレーション

enable_test_federation = 

### DSサーバ
DSのサーバ名を指定します。

 指定しない場合は運用フェデレーションのDSサーバ名がデフォルト値として用いられます。テストフェデレーションに参加している場合は次のセルのコメントを外してDSサーバ名を指定してください。

In [ ]:
# gakunin_ds_hostname = 'test-ds.gakunin.nii.ac.jp'    # テストフェデレーション

### クラウドゲートウェイ

クラウドゲートウェイのサーバ名を指定します。

指定しない場合は運用フェデレーションのサーバ名がデフォルト値として用いられます。テストフェデレーションに参加している場合は次のセルのコメントを外してクラウドゲートウェイのサーバ名を指定してください。

In [ ]:
# cg_fqdn = 'sptest.cg.gakunin.jp'    # テストフェデレーション

### メタデータ検証用証明書
メタデータを検証するための証明書を指定します。

証明書を配布しているURLを次のセルで指定してください。

In [ ]:
# (例)
# metadata_signer_url = 'https://metadata.gakunin.nii.ac.jp/gakunin-signer-2017.cer'          # 運用フェデレーション
# metadata_signer_url = 'https://metadata.gakunin.nii.ac.jp/gakunin-test-signer-2020.cer'  # テストフェデレーション

metadata_signer_url = 

### SimpleSAMLphp

SimpleSAMLphp cronモジュールのトリガー用secret文字列を生成します。

In [ ]:
import random
import string
cron_secret = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
cron_secret

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{target_auth}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

gvars.update({
    'auth_fqdn': auth_fqdn,
    'enable_test_federation': enable_test_federation,
    'metadata_signer_url': metadata_signer_url,
    'cron_secret': cron_secret,
})

for name in ['gakunin_ds_hostname', 'cg_fqdn']:
    if name in vars():
        gvars[name] = vars()[name]
    elif 'name' in gvars:
        del(gvars['name'])

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{target_auth}

## 証明書の配置

IdP-proxyで使用するサーバ証明書、秘密鍵などのファイルを配置します。出どころなどの情報を必要以上に残さないためにNotebookからの操作ではなく、ターミナルなどから**sshでログインして操作を行ってください**。 

配置先となるノードのIPアドレスを確認します。表示されたIPアドレスに対して、ユーザ名`root`と「VCノード作成」のNotebookで設定したSSHの秘密鍵を指定することでログインすることができます。 

In [ ]:
!ansible {target_auth} -c local -m debug -a 'var=servicenet_ip'

 サーバ証明書、秘密鍵をそれぞれ以下のセルの実行結果に表示されるパスに配置してください。

In [ ]:
!ansible {target_auth} -c local -m debug -a 'msg="certs/{{{{auth_fqdn}}}}.cer"'

In [ ]:
!ansible {target_auth} -c local -m debug -a 'msg="certs/{{{{auth_fqdn}}}}.key"'

中間CA証明書がある場合は、サーバ証明書と同じディレクトリに配置してください。中間CA証明書を配置した場合は次のセルにそのファイル名を指定してください。

In [ ]:
# (例)
# intermediate_certfile = 'nii-odca4g7rsa.cer'

intermediate_certfile = 

また中間CA証明書を配置するのではなく、中間CA証明書を連結したファイルを配置することもできます。その場合は次のセルの実行結果に表示されるパスに配置してください。中間CA証明書を連結したファイルが存在している場合は、中間CA証明書を指定しても連結したファイルの方が優先されます。

In [ ]:
!ansible {target_auth} -c local -m debug -a 'msg="certs/{{{{auth_fqdn}}}}.chained.cer"'

**証明書などの配置を行った後に、これ以降の操作を行ってください。**

証明書が配置されたことを確認します。配置されたサーバ証明書の内容を表示してみます。

In [ ]:
!ansible {target_auth} -a \
    'openssl x509 -noout -text -in certs/{{{{auth_fqdn}}}}.cer'

 秘密鍵の内容を表示してみます。

In [ ]:
!ansible {target_auth} -a \
    'openssl rsa -noout -text -in  certs/{{{{auth_fqdn}}}}.key'

中間CA証明書を指定した場合は、そのファイルが配置されていることを確認してgroup_varsに保存します。

In [ ]:
if "intermediate_certfile" in vars():
    !ansible {target_auth} -a \
        'openssl x509 -noout -text -in certs/{intermediate_certfile}'
    import yaml
    from pathlib import Path
    gvars_path = Path(f"group_vars/{target_auth}")
    with gvars_path.open() as f:
        gvars = yaml.safe_load(f)
    gvars["intermediate_certfile"] = intermediate_certfile
    with gvars_path.open(mode='w') as f:
        yaml.safe_dump(gvars, stream=f)

中間CA証明書を連結したファイルを配置した場合は、その内容を確認します。

In [ ]:
!ansible {target_auth} -m shell -a 'chdir=certs \
    test ! -f {{{{auth_fqdn}}}}.chained.cer || \
    ( openssl crl2pkcs7 -nocrl -certfile {{{{auth_fqdn}}}}.chained.cer | openssl pkcs7 -print_certs -noout )'

## IdP-proxyのセットアップ

IdP-proxyコンテナで必要となるファイルを準備する Ansible Playbook を実行します。

ここで実行するplaybookが配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/srv/idp-proxy/docker-compose.yml|コンテナ構成の設定ファイル|
|/srv/idp-proxy/cert/idp-proxy.chained.cer|サーバ証明書(nginx)|
|/srv/idp-proxy/cert/idp-proxy.cer|サーバ証明書(SimpleSAMLphp)|
|/srv/idp-proxy/cert/idp-proxy.key|秘密鍵|
|/srv/idp-proxy/cert/gakunin-signer.cer|メタデータ検証用証明書|

まず、実際に設定を変更する前にドライラン（チェックモード）でPlaybookを実行します。

> ドライランではメタデータを配置するディレクトリが作成されないため、ファイルの配置でエラーとなりますがこの時点では問題ありません。

In [ ]:
!ansible-playbook -l {target_auth} -CDv playbooks/setup-idp-proxy.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_auth} playbooks/setup-idp-proxy.yml

IdP-proxy コンテナのためにセットアップしたディレクトリの状態を確認します。

In [ ]:
!ansible {target_auth} -a 'tree /srv/idp-proxy'

## IdP-proxyコンテナの起動

コンテナイメージを取得します。

In [ ]:
!ansible {target_auth} -a 'chdir=/srv/idp-proxy \
    docker compose pull -q'

IdP-proxyコンテナを起動します。

In [ ]:
!ansible {target_auth} -a 'chdir=/srv/idp-proxy \
    docker compose up -d'

コンテナが起動したことを確認します。

In [ ]:
!ansible {target_auth} -a 'chdir=/srv/idp-proxy \
    docker compose ps'

コンテナのログを確認します。

In [ ]:
!ansible {target_auth} -a 'chdir=/srv/idp-proxy \
    docker compose logs --tail=20'

## 学認にSP設置の申請を行う

申請を行う前に学認（GakuNin）の「[参加情報](https://www.gakunin.jp/join)」にてフェデレーション参加の流れを確認してください。参加するフェデレーションに従い「[テストフェデレーション参加手続き](https://www.gakunin.jp/join/test)」または「[運用フェデレーション参加手続き](https://www.gakunin.jp/join/production)」にある「学認申請システム」から「新規SP申請」を行います。

「新規SP申請」を選択すると以下のような画面が表示されます。
> キャプチャー画面はテストフェデレーションのものです。

![新規SP申請](images/cw-531-01.png)

[IdP-proxy](https://github.com/NII-cloud-operation/CoursewareHub-LC_idp-proxy)では[SimpleSAMLphp](https://simplesamlphp.org/)を利用しています。そのため学認申請システムのテンプレートではなく、構築した IdP-proxyのメタデータを「テンプレート外メタデータ」からアップロードして申請してください。

![メタデータアップロード](images/cw-531-02.png)

申請システムにメタデータをアップロードするために構築したIdP-proxyからメタデータのダウンロードを行います。次のセルを実行することで表示されるリンク先にアクセスするとIdP-proxyのメタデータがダウンロードされます。

In [ ]:
print(f'https://{gvars["auth_fqdn"]}/simplesaml/module.php/saml/sp/metadata.php/default-sp')

ダウンロードしたメタデータを学認申請システムにアップロードすると「SPメタデータ情報」の入力欄のうちメタデータに対応するものが設定されます。他の欄を入力した後に申請してください。ただし CoursewareHubでは `mail` 属性を利用するので「受信する属性情報」に `mail` を**必須** な項目として追加してください。

## 学認mAPとの連携

CoursewareHubでは利用者グループを管理するために[学認mAP](https://meatwiki.nii.ac.jp/confluence/display/gakuninmappublic/Home)を利用します。ここでは IdP-proxyを学認mAPと連携する手順について記します。

### 申請

学認mAPの「[問い合わせ](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=8716731)」などに記されている窓口を通して、構築したSP(IdP-proxy)と学認mAPとを連携するための依頼を行ってください。

### メタデータの登録（テストフェデレーションの場合）

テストフェデレーションの場合、mAP(SP検証環境)のメタデータをIdP-proxyに登録する必要があります。

#### メタデータの指定

mAP利用申請を行うと、学認クラウドゲートウェイサービスサポートからmAP(SP検証環境)のメタデータが送られてきます。送られてきたメタデータを IdP-proxyに登録します。メタデータファイルをこのNotebook環境に配置し、そのパスを次のセルで指定してください。

In [ ]:
# (例)
# cgidp_metadata = './sptestcgidp-metadata.xml'

cgidp_metadata = 

#### パラメータの保存

メタデータファイルのパスをgroup_varsに保存します。

In [ ]:
import yaml
from pathlib import Path
gvars_path = Path(f'group_vars/{target_auth}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)
gvars.update({'cgidp_metadata': str(Path(cgidp_metadata).resolve())})
with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)
    
!cat {gvars_path}

#### 設定変更

メタデータを配置して、変更を反映するためにコンテナの再起動を行います。

ここで実行するplaybookが更新、配置するファイル、ディレクトリを以下の表に示します。

|パス|用途|
|:---|:---|
|/srv/idp-proxy/docker-compose.yml|コンテナ構成の設定ファイル(更新)|
|/srv/idp-proxy/metadata/cgidp-metadata.xml|クラウドゲートウェイのメタデータ|


まず、実際に設定を変更する前にドライラン（チェックモード）でPlaybookを実行します。

In [ ]:
!ansible-playbook -l {target_auth} -CDv playbooks/setup-idp-proxy.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_auth} playbooks/setup-idp-proxy.yml

ファイルを配置したディレクトリの状態を確認します。

In [ ]:
!ansible {target_auth} -a 'tree /srv/idp-proxy'

変更した設定をコンテナに反映させます。

In [ ]:
!ansible {target_auth} -a 'chdir=/srv/idp-proxy \
    docker compose up -d --wait'

コンテナの状態を確認します。

In [ ]:
!ansible {target_auth} -a 'chdir=/srv/idp-proxy \
    docker compose ps'

### SPコネクタの作成

[学認クラウドゲートウェイサービス](https://cg.gakunin.jp/)またはSP検証環境(テストフェデレーションの場合)にアクセスしてSPコネクタの作成を行います。手順の詳細については学認クラウドゲートウェイサービス利用マニュアルの「[SPコネクタを作成する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873902)」の章を参照してください。

> テストフェデレーションSP検証環境のアドレスは利用申請後にメールなどで通知されます。

SP管理者が学認クラウドゲートウェイのウェブサイトにアクセスすると以下のような画面が表示されます。

> キャプチャー画面はテストフェデレーションのものです。

![クラウドゲートウェイ画面](images/cw-531-03.png)

「SPコネクタの作成」を選択すると入力画面が表示されるので、必要な項目を入力してください。

![新規SPコネクタ入力画面](images/cw-531-04.png)

詳細設定の「Q1 作成するSPコネクタは、検索対象にしますか？」との項目で「検索対象としない」を選択した場合は、グループ管理者がSPコネクタを見つけることが出来なくなります。グループとの接続を行うためにはSPコネクタ側からグループへ接続申請をする必要があります。詳細については学認クラウドゲートウェイサービス利用マニュアルの「[SPコネクタにグループを接続する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873907)」を参照してください。